In [ ]:
!pip install keras-ocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 957 kB 11.9 MB/s 
     |████████████████████████████████| 604 kB 42.2 MB/s 
     |████████████████████████████████| 9.5 MB 27.6 MB/s 
     |████████████████████████████████| 50 kB 8.5 MB/s 
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=71c6533312a6b8edacb40d10c217ec6140c1d733fb238514d6c9940055c66807
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca7b1f68da6d94252bb6a8d6ad4f18e04e9
Successfully built validators


In [ ]:
import cv2
import numpy as np
import pandas as pd
import re
import spacy
import en_core_web_sm
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import keras_ocr
pipeline = keras_ocr.pipeline.Pipeline()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5


In [ ]:
!gdown 1JR3Sgz2-AyfIDT7jYRXL8XMrMom57DAi
!gdown 13du4J0NVPa6NWcehJITx1jcfYsINdBE5

Downloading...
From: https://drive.google.com/uc?id=1JR3Sgz2-AyfIDT7jYRXL8XMrMom57DAi
To: /content/english_dataset.csv
100% 643k/643k [00:00<00:00, 126MB/s]
Downloading...
From: https://drive.google.com/uc?id=13du4J0NVPa6NWcehJITx1jcfYsINdBE5
To: /content/Allergen.csv
100% 2.05k/2.05k [00:00<00:00, 3.31MB/s]


In [ ]:
df = pd.read_csv('english_dataset.csv')
aller_list = pd.read_csv('Allergen.csv') 

In [ ]:
df = df.drop(414)

In [ ]:
df.head()

,ingredients_text,allergens,image_ingredients_url,image_nutrition_url,is_allergen,ingredeint_language
0,"water, _almonds_* 5%, agave syrup*, sea salt",en:nuts,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,True,en
1,"Wheat flour, sugar, almonds 18%, eggs, egg Whi...","en:eggs,en:gluten,en:milk,en:nuts,en:soybeans",https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,True,en
2,"_WHEAT_ flour, water, tomato pulp, mozzarella ...","en:gluten,en:milk",https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,True,en
3,LATTE_ INTERO BIOLOGICO,en:milk,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,True,en
4,"Pistachio 45%, sugar, milk powder, extra virgi...","en:milk,en:nuts,en:soybeans",https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,True,en


In [ ]:
nlp = en_core_web_sm.load()

In [ ]:
aller_list.head()

,Ingridients
0,celery
1,lupin
2,lupine
3,milk
4,cheese


In [ ]:
np.set_printoptions(precision=2)

In [ ]:
allergen_embeddings = []
for allergen in aller_list.Ingridients:
  vector = nlp(allergen)
  allergen_embeddings.append(vector.vector)

In [ ]:
def preprocess(text):
    text = re.sub('#S+', '',text)
    text = re.sub('@S+', '  ',text)
    text = re.sub('httpS+s*', ' ', text) 
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+-,./:;<=>?@[]^_`{|}~"""), ' ', text)
    text = re.sub('RT|cc', ' ', text)
    text = re.sub(r'<[^<]+?>', '', text)
    text = re.sub(r'[0-9]', "", text)
    text = text.replace('\n', ' ') 
    text = re.sub(r'\s+', ' ', text)
    return text

In [ ]:
def detect_allergen(image_url):
  allergen = []
  final_ocr = []
  image = keras_ocr.tools.read(image_url)
  prediction_group = pipeline.recognize([image])
  for text in prediction_group[0]:
    final_ocr.append(text[0])
  final_ocr = " ".join(str(x) for x in final_ocr)
  clean_ocr = preprocess(final_ocr)
  clean_ocr = word_tokenize(clean_ocr)
  clean_ocr = [i.lower() for i in clean_ocr]
  print(list(clean_ocr))
  for i in clean_ocr:
    for j in allergen_embeddings:
      ingred = nlp(i)
      ingred = np.array(ingred.vector.reshape(1, -1))
      aller = np.array(j).reshape(1, -1)
      sim = cosine_similarity(ingred, aller)
      if sim > 0.90:


        print('May contain {}'.format(i))

In [ ]:
detect_allergen(df.image_ingredients_url[10])

['gb', 'biscuits', 'filed', 'cappucaino', 'cream', 'hond', 'with', 'sunfaner', 'vegelable', 'whole', 'ingrediets', 'wheat', 'nour', 'sugu', 'vegetable', 'cls', 'of', 'palm', 'and', 'palm', 'oil', 'wsten', 'mugonne', 'harelnut', 'eggs', 'butter', 'whey', 'powder', 'coffee', 'isn', 'the', 'sin', 'wheat', 'stanch', 'glucose', 'pie', 'n', 'mill', 'powcer', 'snp', 'sodum', 'carbonaes', 'flvounegs', 'imay', 'coca', 'emuliher', 'ledthin', 'sall', 'rasing', 'sgents', 'and', 'ammoowm', 'geam', 'soy', 'sounes', 'dry', 'from', 'ught', 'best', 'tefore', 'eno', 'fats', 'keep', 'ool', 'and', 'away', 'no', 'hydrogented', 'contain', 'peanut', 'wraes', 'wade', 'ialy', 'bclon', 'in', 'date', 'prinited', 'sce']
May contain cream
May contain wheat
May contain butter
May contain wheat
May contain soy
May contain peanut


In [ ]:
detect_allergen(df.image_ingredients_url[10])